In [1]:
import networkx as nx
import sqlite3
import pandas as pd

conn = sqlite3.connect('/Users/Admin/Documents/Projects/Kairos/main/data/GTFS.db')

#### Load DataFrames

In [2]:
stops = pd.read_sql_query('SELECT * FROM gtfs_stops', conn)
stops_time = pd.read_sql_query('SELECT * FROM gtfs_stop_times', conn)
frequencies = pd.read_sql_query('SELECT * FROM gtfs_frequencies', conn)
stops_time_first = stops_time[stops_time['stop_sequence'] == 1].reset_index(drop=True)

In [4]:
MG = nx.MultiDiGraph() # initialise Graph

In [7]:
stops_time.head(1)

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign,pickup_type,drop_off_type,shape_dist_traveled,timepoint,arrival_time_seconds,departure_time_seconds
0,1012-10-0,03:40:00,03:40:00,301708,1,None,None,None,None,None,13200,13200


In [8]:
stops.head(1)

,stop_id,stop_name,stop_desc,stop_lat,stop_lon,zone_id,stop_url,stop_code,stop_street,stop_city,stop_region,stop_postcode,stop_country,location_type,parent_station,stop_timezone
0,18848,Clínicas,None,-23.55419,-46.670723,None,None,None,None,None,None,None,None,None,None,None


In [5]:
## Add Nodes

In [7]:
for i, row in stops.iterrows():
    MG.add_node(row['stop_id'], location={'long': row['stop_lon'], 'lat': row['stop_lat']})

In [8]:
## Add Edges

In [9]:
def unicode_to_seconds(a):
    a = map(int, a.split(':'))
    return a[0] * 3600 + a[1] * 60 + a[2]

In [11]:
for i, row in stops_time.iterrows():
    
    try:
        partida, chegada = stops_time.iloc[i], stops_time.iloc[i + 1]
    except:
        break
    if chegada['trip_id'] == partida['trip_id']:
        MG.add_edge(partida['stop_id'], chegada['stop_id'], 
                    attr = {'tempo': chegada['departure_time_seconds']-partida['arrival_time_seconds'],
                      'trip_id': chegada['trip_id']}
        )

## Add frequency

In [16]:
frequencies.head(2)

,trip_id,start_time,end_time,headway_secs,start_time_seconds,end_time_seconds
0,1012-10-0,03:00:00,03:59:00,1800,10800,14340
1,1012-10-0,04:00:00,04:59:00,420,14400,17940


In [17]:
stops_time.head(2)

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign,pickup_type,drop_off_type,shape_dist_traveled,timepoint,arrival_time_seconds,departure_time_seconds
0,1012-10-0,03:40:00,03:40:00,301708,1,None,None,None,None,None,13200,13200
1,1012-10-0,03:42:00,03:42:00,30003039,2,None,None,None,None,None,13320,13320


In [28]:

stops_time_first.head(2)

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign,pickup_type,drop_off_type,shape_dist_traveled,timepoint,arrival_time_seconds,departure_time_seconds
0,1012-10-0,03:40:00,03:40:00,301708,1,None,None,None,None,None,13200,13200
1,1012-10-1,00:50:00,00:50:00,301729,1,None,None,None,None,None,3000,3000


### Primeiros Stops

In [33]:
for i, row in stops_time_first.iterrows():
    
    
    if i % 100==0:
        print('id', i)

    lista = []
    lista.append(row['arrival_time_seconds'])
    trip = frequencies[frequencies['trip_id'] == row['trip_id']]
    
    rodou = 0
    fullday = 0
    
    if len(trip[(trip["start_time_seconds"] <= lista[0]) &
                             (trip["end_time_seconds"] + 60 >= lista[0])]) > 0:
    
        # Para baixo
        while 1:

            ultimo = lista[-1] 
            timetable = trip[(trip["start_time_seconds"] <= ultimo) &
                             (trip["end_time_seconds"] + 60 >= ultimo)]

            if len(timetable) > 0:
                
                soma = timetable['headway_secs'].values[0] + ultimo
                
                if rodou == 1 and soma > lista[0]:
                    fullday = 1
                    print('fullday')
                    break
                
                else:
                    if soma <= 86340:
                        lista.append(soma)
                    else: 
                        rodou = 1
                        lista.append(soma - 86340)
            
            else:
                break


        # Para cima  
        while fullday==0:

            primeiro = lista[0]
            timetable_atual = trip[(trip["start_time_seconds"] <= primeiro) 
                                   & (trip["end_time_seconds"] + 60 >= primeiro)]

            if timetable_atual['start_time_seconds'].values[0] == 0:
                timetable_anterior = trip[trip['end_time_seconds'] == 82740]
            else:
                timetable_anterior = trip[(trip['end_time_seconds']) == 
                                          timetable_atual['start_time_seconds'].values[0] - 60]

            # com timetable anterior
            if len(timetable_anterior) > 0:

                subt = primeiro - timetable_anterior['headway_secs'].values[0]
                if subt < 0:
                    subt = subt + 82740

                # se dentro da timetable anterior
                if subt <= timetable_anterior['end_time_seconds'].values[0]:
                    lista.insert(0, subt) 
                else:
                    lista.insert(0, primeiro - timetable_atual['headway_secs'].values[0])

            # se dentro da timetable atual mas timetable anterior inexistente 

            elif ((primeiro - timetable_atual['headway_secs'].values[0]) >=
                    timetable_atual['start_time_seconds'].values[0]):

                lista.insert(0, primeiro - timetable_atual['headway_secs'].values[0])

            else:
                break


        if 'viagens' in MG.node[row['stop_id']]:
            MG.node[row['stop_id']]['viagens'][row['trip_id']] = lista
        else:
            MG.node[row['stop_id']]['viagens'] = {row['trip_id']: lista}

    else:
        print('pulou')

id 0
id 100
id 200
id 300
id 400
id 500
id 600
id 700
id 800
id 900
id 1000
id 1100
id 1200
id 1300
id 1400
id 1500
id 1600
pulou
id 1700
fullday
fullday
id 1800
id 1900
pulou
id 2000
id 2100
id 2200
id 2300


In [2]:
nx.write_gexf(MG, 'graphs/sp_partial2.gexf')

NameError: name 'nx' is not defined

#### checar se frequencies tem linhas com descontinuidade nos horários

for i, row in stops_time_first.iterrows():
    
    trip = row['trip_id']
    
    linha = frequencies[frequencies['trip_id'] == row['trip_id']]
    
    
    
    for i, hora in linha.iterrows():
        
        try:
            linha.iloc[i+1]
        except:
            break
        
        start = linha.iloc[i+1]['start_time_seconds']
        
        end = linha.iloc[i]['end_time_seconds']
        
        
        
        if not(start - end == 60) and i < (len(linha) - 1):
            print('oi')   

### READ FILE

In [3]:
import ast
MG = nx.read_gexf('graphs/sp_partial2.gexf')graphs/

In [4]:
for node in MG.nodes(data=True):

    for key, value in MG.edge[node[0]].items():
        for k, v in value.items():
            if type(v['attr']) == str:
                MG.edge[node[0]][key][k]['attr'] = ast.literal_eval(v['attr'])

In [5]:
for node in MG.nodes(data=True):
    if type(MG.node[node[0]]['location']) == str:
        pass
        # MG.node[node[0]]['location'] = ast.literal_eval(node[1]['location'])
    if len(node[1]) == 3:
        if type(MG.node[node[0]]['viagens']) == str:
            MG.node[node[0]]['viagens'] = ast.literal_eval(MG.node[node[0]]['viagens'])


### Resto dos stops
- Caminho fechado busao

In [55]:
for i, row in stops_time_first.iterrows():
       
    if i % 100 == 0:
        print(i)
        
    trip_id = row['trip_id']
    stop_id_velho = row['stop_id']
    
    outputs_set = []
    j = 0
    
    while 1:
        
        output = getpath(MG.edge[stop_id_velho], trip_id) # get path in edge dict to stop_id
        
        if output == None or output in outputs_set: # if no path or already visited, break
            break            
        else:
            outputs_set.append(output)
        
        stop_id_novo, number, *oi = output # distribuir variáveis
        
        
        # get previous timetable
        try:
            timetable = MG.node[stop_id_velho]['viagens'][trip_id]
        except KeyError:
            print(stop_id_velho)
            continue
        
        # get trip time
        trip_time = MG.edge[stop_id_velho][stop_id_novo][number]['attr']['tempo']
        
        # generate new timetable
        newtimetable = [i + trip_time for i in MG.node[stop_id_velho]['viagens'][trip_id]]

        # append new timetable to stop
        if 'viagens' in MG.node[stop_id_novo]:
            MG.node[stop_id_novo]['viagens'][trip_id] = newtimetable
        else:
            MG.node[stop_id_novo]['viagens'] = {trip_id: newtimetable}
        
        # replace stop_ids
        stop_id_velho = stop_id_novo
        
        j = j + 1

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
6311345
1700
1800
1900
130001844
2000
2100
2200
2300


In [8]:
def getpath(nested_dict, value, prepath=()):
    for k, v in nested_dict.items():
        path = prepath + (k,)
        if v == value: # found value
            return path
        elif hasattr(v, 'items'): # v is a dict
            p = getpath(v, value, path) # recursive call
            if p is not None:
                return p

In [58]:
nx.write_gexf(MG, 'graphs/sp_final.gexf')